In [1]:
from pelux_core import *

Could not import regions, which is required for some of the functionalities of this module.


# Catalogue queries
You can search for coordinates and photometry of a list of stars:

In [2]:
my_file=r'C:\Users\Vito\Desktop\PhD\Programmi\Python\prove\target_list_coordinates.csv'
import_phot(my_file,coordinates=True)

C:\Users\Vito\AppData\Local\Programs\Python\Python38\lib\site-packages\tabulate.py:595: UserWarning: Warning: converting a masked element to nan.
  conv(string)
C:\Users\Vito\AppData\Local\Programs\Python\Python38\lib\site-packages\tabulate.py:996: UserWarning: Warning: converting a masked element to nan.
  return format(float(val), floatfmt)


The file can be either a list of equatorial coordinates (RA, dec), with header 'ra_v, dec_v' (.txt or .csv allowed), or a list of star names (no header, only .txt allowed). In the former case, the keyword coordinates must be set to 'True'.

In default mode, the used catalogues are the following:
- Gaia EDR3;
- 2MASS.

To specify a different set of catalogues, you can use the keyword surveys:

In [4]:
import_phot(my_file,coordinates=True,surveys=['GAIA_EDR3','ALLWISE'])